# ExplorerLand data
https://explorer.land

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Polygon, MultiPolygon, LineString, MultiLineString, Point, MultiPoint
import json
import requests
import time
from random import uniform

In [3]:
projects = pd.DataFrame()
for page in range(0, 10):
    url = f"https://api.explorer.land/v1/public/projects?listed=1&status_ids[]=e6b4q7jev3n8xd50&status_ids[]=exm7vzjapj0r5846&order_by_status=1&include=activity_types,ecosystem_types,partners,tabs,countries,modules&limit=1000&page={page}"
    response = requests.get(url).json()
    projects = pd.concat([projects, pd.json_normalize(response['data'], sep='_')], ignore_index = True)

In [4]:
projects = projects.rename(columns = {'slug':'project_id_reported','long_description':'project_description_reported'})

In [5]:
projects

,object,id,project_id_reported,name,slogan,description,project_description_reported,title,video_url,video_thumbnail_url,...,sponsorship_sponsor_count,funding_opportunities_count,geojson_type,geojson_coordinates,partners_data,activity_types_data,ecosystem_types_data,modules_data,tabs_data,countries_data
0,Project,9xdbkywvgdjogwmz,fwasmarketplace,"The ""Fish With A Story"" MARKETPLACE Platform",Fish with a social and ecological story: rethi...,"""Fish With A Story"" – by ABALOBI – is fully tr...","# ""Fish with a Story""\r\nA Story on a plate .....",QrpxX7BfHF84e9cehqNdq9rv3J4NcbtLwtIP5gm6.jpg,https://youtube.com/embed/yOqBbFCU344,https://img.youtube.com/vi/yOqBbFCU344/hqdefau...,...,0,0,Point,"[18.813381, -34.102679]","[{'object': 'Partner', 'id': '0waln3ko5bj4bg5r...","[{'object': 'Tag', 'id': 'ow54el3p0ly9znaq', '...","[{'object': 'Tag', 'id': 'opxazw3ggl3dlmr4', '...",[],"[[forests-forward], [about], [news], [goods], ...","[{'id': 'x7qrdn37da34abmz', 'name': 'South Afr..."
1,Project,x7qrdn37ma34abmz,corridors_oued_tine,Corridors Caroubier ''ممرات خروب كابت كربون تغ...,Ici on plante du caroubier pour le climat ! هن...,"Le projet C6 ""Corridor Caroubier CAPTE Carbone...",L’ambition de Cx6 est de contribuer à la struc...,JGah6hlCQVtB7SBVFwTK9jRXd2ovnitsQJNBEgpz.jpg,https://youtube.com/embed/2vCu93ArkXA,https://img.youtube.com/vi/2vCu93ArkXA/hqdefau...,...,0,0,Point,"[9.714216, 36.894821]","[{'object': 'Partner', 'id': 'vg0drw387myx4z9q...","[{'object': 'Tag', 'id': 'xabwdoj4eoykm8zg', '...","[{'object': 'Tag', 'id': 'lwn9v8ydxejz0xrm', '...","[{'name': 'project_commodity_sales', 'legacy_n...","[[forests-forward], [about], [news], [sponsors...","[{'id': '6qorwg35rbj04kem', 'name': 'Tunisia (..."
2,Project,e6b4q7jevv3n8xd5,vcm,Gula Gula Food Forest Programs: Carbon Offsett...,"Restoration for livelihoods, biodiversity and ...",The Food Forest Program aims at restoring fore...,__*Every action counts. You can directly suppo...,lcuj0xsmUmefb6WEL3jRUYvu8GtRSOAIG22CBhQK.jpg,None,None,...,0,0,Point,"[100.488625, -0.453352]","[{'object': 'Partner', 'id': 'v9nz6b3v9vy8eaql...","[{'object': 'Tag', 'id': 'opxazw3gvw3dlmr4', '...","[{'object': 'Tag', 'id': 'lwn9v8ydxejz0xrm', '...","[{'name': 'project_commodity_sales', 'legacy_n...","[[forests-forward], [about], [news], [sponsors...","[{'id': 'e6b4q7jeevjn8xd5', 'name': 'Indonesia..."
3,Project,pxazw3g904ydlmr4,reforestation-,Restoration in magamba forest reserve.,Reforestation and agroforestry project,Green Land Group is the organization which dea...,### The project is well dedicating to remove p...,XY7hC3XLbbbbeNzJOnqP4McHYH2JRCiJYGBDTWII.jpg,None,None,...,0,0,Point,"[38.313701, -4.774361]","[{'object': 'Partner', 'id': 'abwdoj4p0ljkm8zg...","[{'object': 'Tag', 'id': 'xabwdoj4eoykm8zg', '...","[{'object': 'Tag', 'id': 'lwn9v8ydxejz0xrm', '...","[{'name': 'project_statistics', 'legacy_name':...","[[forests-forward], [about], [news], [goods], ...","[{'id': 'pq4d7myr8gja6kzx', 'name': 'Tanzania ..."
4,Project,wzgbdynn8wykae0p,rewilding_wusseken,eMission-X: Rewilding Wusseken,"Rewilding: Revive Nature, Restore Balance!",We buy land from industrial agriculture and le...,Rewilding in Wusseken by eMission-X gGmbH\r\n\...,Twg6RDfON4zzHSQBMIQYb9jyPdKNySczkyi3ncVq.jpg,https://youtube.com/embed/hBCMNqi7-wM,https://img.youtube.com/vi/hBCMNqi7-wM/hqdefau...,...,0,0,Point,"[13.674201, 53.763913]","[{'object': 'Partner', 'id': 'lmedq3qex8397va4...","[{'object': 'Tag', 'id': '7md4563zqk3kno8b', '...","[{'object': 'Tag', 'id': 'lwn9v8ydxejz0xrm', '...",[],"[[forests-forward], [about], [news], [goods], ...","[{'id': 'pq4d7myrd53a6kzx', 'name': 'Germany (..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,Project,9nz6b3v7gb38eaql,gatopreto,SDGs Families,None,We have worked with communities of Lake Distri...,Mozambique is one of the countries most affect...,s2rg1x98xsCYynSFdjRIm7ZtL5HkN7EWpCHL37Wz.jpg,None,None,...,0,0,Point,"[34.689098, -12.439639]","[{'object': 

In [6]:
sites = pd.DataFrame()
for project in projects.project_id_reported.unique().tolist():
    url = f"https://api.explorer.land/v1/public/spots/polygon?project_slug={project}"
    response = requests.get(url).json()
    if response['features']:
        temp = pd.json_normalize(response['features'], sep='_')
        temp['project_id_reported'] = project
        sites = pd.concat([sites, temp], ignore_index = True)
        time.sleep(uniform(0, 5.0))

In [7]:
raw_df = sites.copy()

In [8]:
raw_df["geometry"] = raw_df["geometry_coordinates"].apply(lambda coords: MultiPolygon([Polygon(p[0]) for p in coords]))

In [9]:
gdf = gpd.GeoDataFrame(raw_df, geometry='geometry', crs = 'EPSG:4326')

### Fix geometries

Combine linestrings to polygons, if applicable

In [10]:
gdf['geometry'] = gdf['geometry'].apply(
    lambda geom: Polygon(list(geom.coords) + [geom.coords[0]]) if isinstance(geom, LineString) and not geom.is_closed and len(geom.coords) > 0 else
                 Polygon(geom.coords) if isinstance(geom, LineString) and geom.is_closed else
                 geom
)

Explode multipolygons to individual polygons

In [11]:
while 'MultiPolygon' in gdf.geometry.geom_type.unique().tolist():
    gdf = gdf.explode(index_parts=False)
    gdf['geometry'] = gdf['geometry'].make_valid()

Keep only polygons and points

In [12]:
gdf = gdf.loc[gdf.geometry.geom_type.isin(['Polygon', 'Point'])].copy()

Keep only reforestation sites

In [13]:
gdf = gdf.loc[gdf.properties_category_label == 'Reforestation']

Create site-level id

In [14]:
gdf = gdf.reset_index(drop = True).reset_index(names=['site_id_created'])

Add project-level description

In [15]:
gdf = (gdf
       .loc[:, ['site_id_created', 'properties_slug', 'project_id_reported', 'properties_description', 'geometry']]
       .rename(columns = {'properties_slug':'site_id_reported','properties_description':'site_description_reported'})
       .merge(projects[['project_id_reported', 'project_description_reported']], on = 'project_id_reported', how = 'left')
       .drop_duplicates()
       .reset_index(drop = True))

### Fix geometries

In [16]:
invalid_geom = False

if len(gdf[gdf.geometry.is_valid == False]) > 0:
    invalid_geom = True    

### Harmonize nomenclature

In [17]:
gdf['host_name'] = 'Explorer Land'
gdf['url'] = 'https://explorer.land/x/project/' + gdf.project_id_reported + '/site/' + gdf.site_id_reported + '/about'
gdf['site_sqkm'] = gdf['geometry'].to_crs('EPSG:3857').area / 1e6
gdf['project_geometries_invalid'] = invalid_geom

In [18]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3091 entries, 0 to 3090
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   site_id_created               3091 non-null   int64   
 1   site_id_reported              3091 non-null   object  
 2   project_id_reported           3091 non-null   object  
 3   site_description_reported     1821 non-null   object  
 4   geometry                      3091 non-null   geometry
 5   project_description_reported  3089 non-null   object  
 6   host_name                     3091 non-null   object  
 7   url                           3091 non-null   object  
 8   site_sqkm                     3091 non-null   float64 
 9   project_geometries_invalid    3091 non-null   bool    
dtypes: bool(1), float64(1), geometry(1), int64(1), object(6)
memory usage: 220.5+ KB


In [19]:
gdf["project_geometries_invalid"].value_counts()

project_geometries_invalid
False    3091
Name: count, dtype: int64

### Save it

In [20]:
gdf.to_file("../midsave/explorer_land.gpkg")